# Reinforcement Learning in Mario - Tutorial

This tutorial will gude you through the setup process for the Mario Environment and Stablebaselines library

## First steps

- Create a conda virtual environment with Python 3.9
    - Make sure you have anaconda installed in your computer - (Conda Installation)[https://conda.io/projects/conda/en/latest/user-guide/install/index.html]
    -  Open the Conda console and use the following line to create the virtual environment: 
        ```console
        conda create --name my_environment python=3.9
        conda activate my_environment
        ```
        *Note the new environment only has Python, we will install the dependencies along the way to detect potential compatibility issues.

- Setup the VScode IDE to run Python from the new environment
    - Open the command palette, use the following shortcut: Control + Shift + P or under the menu "view."
    - Type in the textbox "select interpreter" and choose the python from the new virtual environment.
    - For further information about this process, follow the link: (Python environments in VS Code)[https://code.visualstudio.com/docs/python/environments]

- As next step, please create a private repository in GitHub and share it with (@dccartagena)[https://github.com/dccartagena]. Then clone it in your computer and save everything there. 
    - If you are not familiar with GitHub, I recommend you to use the visual interface to pull/push/fetch changes in your code: (GitHub Desktop)[https://desktop.github.com/]

- Now, let us proceed to create a Jupyter notebook or python file, e.g. mario_saferl.ipynb or mario_saferl.py

## Installing The Mario Environment

- Make sure you are working in the virtual environment. You may check it on the Conda console.
- Run the following commands in the Conda console:
    ```console 
    pip install nes-py
    pip install gym-super-mario-bros
    ```
    - In case you face a compatibility issue with nes-py, chances are that the C++ distribution in your pc is not compatible. Thus, follow the instruction in the first comment of this Stackoverflow question: (How to solve "error: Microsoft Visual C++ 14.0 or greater is required" when installing Python packages?)[https://stackoverflow.com/questions/64261546/how-to-solve-error-microsoft-visual-c-14-0-or-greater-is-required-when-inst]
        - Go to the control panel in your pc and under the *"Install/Uninstall Apps,"* search for the program *"Visual Studio Build Tools,"* then click in **Modify** the installation. 